In [1]:
using DataFrames
using Gadfly
using Cairo
using Compose
using Distributions
using Formatting
set_default_plot_size(40cm, 14cm)

In [2]:
function meta_var(s::AbstractString,v::Any)
         s=symbol(s) 
         @eval (($s) = ($v))
end

meta_var (generic function with 1 method)

In [3]:
function tognm(vtor_anm)
    vtor_gnm = Array{Float64}(convert(Int64, length(vtor_anm)/3));
    vtor_anm =  vtor_anm.^2
    for i=1:convert(Int64, length(vtor_anm)/3)
        vtor_gnm[i] = sqrt(vtor_anm[i*3-2] + vtor_anm[i*3-1] + vtor_anm[i*3])
    end
    return vtor_gnm
end

tognm (generic function with 1 method)

In [4]:
function read_ptraj_modes(file, modes_elements, norma::Bool=true)
    modes_file=open(file, "r")
    modes_text = readdlm(modes_file, skipstart=0, skipblanks=true, 
    ignore_invalid_chars=true, comments=true, comment_char='\*')
    close(modes_file)

    nmodes = modes_text[1, 5]
    ncoords = convert(Int64, modes_elements)
    lines = ceil(Int64, ncoords/7)
    rest = convert(Int64, ncoords % 7)
    
    eval=Array{Float64}(nmodes);
    mode = Array{Float64}(ncoords, nmodes);
    temp1=Array{Float64}(ncoords, 1);
    temp2 = Array{Float64}(ncoords+(7-rest));

    j=lines + 1 + 2 # 1 p/ q lea la prox linea 2 por el header

    for i=1:nmodes
        eval[i] = modes_text[j, 2]
        temp = transpose(modes_text[(j+1):(lines+j), :])
        temp2 = reshape(temp, ncoords+(7-rest))
        for k=(rest+1):7
            pop!(temp2)
        end
    mode[:, i] = temp2
        j = j + lines + 1
    end
    
    if norma == true
        for i=1:nmodes
            mode[: ,i] = mode[:, i] / norm(mode[:, i])
        end
    end
    
    return mode, eval
end

read_ptraj_modes (generic function with 2 methods)

In [5]:
function WeightedHist(in_vec, in_bins, in_weight, density = false)
    # Safety check    
    if length(in_vec) != length(in_weight)
        println("Each element of the input vector needs one weight")
        return
    end
    
    # Prepare variables
    out_counts = Array{Float64}(length(in_bins)-1)
    
    # Get weighted histogram
    for i=1:length(in_bins)-1
        temp_bool = (in_vec .>= in_bins[i]) & (in_vec .< in_bins[i+1])
        out_counts[i] = sum(in_weight[temp_bool])
    end
    
    # Get bins middle points
    out_middle = (in_bins[1:end-1] + in_bins[2:end]) / 2
    
    # Turn counts into density
    if (density == true)
        out_counts = out_counts ./ sum(out_counts) 
    end
    return out_counts, out_middle
end

WeightedHist (generic function with 2 methods)

In [6]:
function read_rst7(file, velocities = true, box_dims = true, box_angles = true)
    # Read
    rst7_file=open(file, "r")
    rst7_text = readdlm(rst7_file, skipstart=1)
    close(rst7_file)

    # Get ready
    natoms = rst7_text[1, 1]
    time_ps = rst7_text[1, 2]
    nrows = convert(Int64, ceil(natoms / 2))

    coords = Array{Float64}(3, natoms)
    vels = Array{Float64}(3, natoms)
    bdims = Array{Float64}(3)
    bangles = Array{Float64}(3)

    # Get coordinates
    orig_coords = rst7_text[2:(nrows+1), :]
    for i=1:nrows
        coords[:, (i*2-1)] = reshape(orig_coords[i, 1:3], 3)
        try
            coords[:, (i*2)] = reshape(orig_coords[i, 4:6], 3)
        end
    end

    # Get velocities?
    if velocities
        try
            orig_vels = rst7_text[(nrows+2):(end-1), :]
            for i=1:nrows
                vels[:, (i*2-1)] = reshape(orig_vels[i, 1:3], 3)
                try
                    vels[:, (i*2)] = reshape(orig_vels[i, 4:6], 3)
                end
            end
        catch
            error("Could not read velocities")
        end
    end

    # Get box size?
    if box_dims
        try
            bdims = convert(Array{Float64}, rst7_text[end, 1:3])
        catch
            error("Could not read box dimensions")
        end
    end

    # Get box angles?
    if box_angles
        try
            bangles = convert(Array{Float64}, rst7_text[end, 4:6])
        catch
            error("Could not read box angles")
        end
    end

    return coords, vels, bdims, bangles, natoms
end

read_rst7 (generic function with 4 methods)

In [8]:
function read_amber_crd(filename_top, filename_rst7, velocities = true, box_dims = true, box_angles = true)
    # Read topology
    top_file=open(filename_top, "r")
    top_text = readdlm(top_file)
    close(top_file)

    # Look for the relevant flags
    match_sp = convert(Int64, find(x -> x == "SOLVENT_POINTERS", top_text[:, 2])[1])
    match_apm = convert(Int64, find(x -> x == "ATOMS_PER_MOLECULE", top_text[:, 2])[1])
    match_rl = convert(Int64, find(x -> x == "RESIDUE_LABEL", top_text[:, 2])[1])
    match_an = convert(Int64, find(x -> x == "ATOM_NAME", top_text[:, 2])[1])
    sum_natoms_all = convert(Int64, top_text[7, 1])

    # Start off with solvent pointers
    # Final residue that is considered part of the solute
    iptres = top_text[match_sp + 2, :][1]
    # Total number of molecules
    nspm = top_text[match_sp + 2, :][2]
    # The 1st solvent molecule
    nspsol = top_text[match_sp + 2, :][3]

    # Now, Atoms per molecule flag
    natoms_each_mol_str = top_text[match_apm + 2, :]
    natoms_each_mol = [0]
    sum_natoms = 0
    for i=1:nspsol-1
        natom_mol_str = natoms_each_mol_str[i]
    
        if natom_mol_str != ""
            each = convert(Int64, natom_mol_str)
            sum_natoms = sum_natoms + each
            push!(natoms_each_mol, sum_natoms)
        else
            break
        end
    end
        
    # Read coordinates
    rst7_file=open(filename_rst7, "r")
    rst7_text = readdlm(rst7_file, skipstart=1)
    close(rst7_file)

    # Get ready
    natoms = rst7_text[1, 1]
    time_ps = rst7_text[1, 2]
    nrows = convert(Int64, ceil(natoms / 2))

    coords = Array{Float64}(3, natoms)
    vels = Array{Float64}(3, natoms)
    bdims = Array{Float64}(3)
    bangles = Array{Float64}(3)

    # Get coordinates
    orig_coords = rst7_text[2:(nrows+1), :]
    for i=1:nrows
        coords[:, (i*2-1)] = reshape(orig_coords[i, 1:3], 3)
        try
            coords[:, (i*2)] = reshape(orig_coords[i, 4:6], 3)
        end
    end
    # One Array{Float64, 2} for each molecule, and the last one for water
    sys_coords = Array{Array{Float64, 2}}(length(natoms_each_mol))
    for i = 1:length(natoms_each_mol)
        lo_margin = natoms_each_mol[i] + 1
        try
            hi_margin = natoms_each_mol[i+1]
            sys_coords[i] = coords[:, lo_margin:hi_margin]
        catch
            sys_coords[i] = coords[:, lo_margin:end]
        end
    end


    # Get velocities?
    if velocities
        try
            orig_vels = rst7_text[(nrows+2):(end-1), :]
            for i=1:nrows
                vels[:, (i*2-1)] = reshape(orig_vels[i, 1:3], 3)
                try
                    vels[:, (i*2)] = reshape(orig_vels[i, 4:6], 3)
                end
            end
        catch
            error("Could not read velocities")
        end
    end
    # One Array{Float64, 2} for each molecule, and the last one for water
    sys_vels = Array{Array{Float64, 2}}(length(natoms_each_mol))
    for i = 1:length(natoms_each_mol)
        lo_margin = natoms_each_mol[i] + 1
        try
            hi_margin = natoms_each_mol[i+1]
            sys_vels[i] = vels[:, lo_margin:hi_margin]
        catch
            sys_vels[i] = vels[:, lo_margin:end]
        end
    end

    # Get box size?
    if box_dims
        try
            bdims = convert(Array{Float64}, rst7_text[end, 1:3])
        catch
            error("Could not read box dimensions")
        end
        bdims = reshape(bdims, length(bdims))
    end

    # Get box angles?
    if box_angles
        try
            bangles = convert(Array{Float64}, rst7_text[end, 4:6])
        catch
            error("Could not read box angles")
        end
        bangles = reshape(bangles, length(bangles))
    end

    # Get number of atoms of each system
    nnatoms_each_mol = Array{Int64}(length(natoms_each_mol))
    for i = 1:length(nnatoms_each_mol)
        nnatoms_each_mol[i] = size(sys_coords[i])[2]
    end
    
    # Finally atom names, again from .prmtop
    top_file=open(filename_top, "r")
    nlines_an = convert(Int64, ceil(sum_natoms_all / 20))
    testeo = AbstractString
    cnt = 0
    atom_names = Array{Array{AbstractString, 1}}(length(nnatoms_each_mol))
    atom_names_full = Array{AbstractString, 1}((nlines_an-1)*20)

    for linea in eachline(top_file)
        try
            testeo = linea[1:15]
        end
        if testeo == "%FLAG ATOM_NAME"
            for linea in eachline(top_file) 
                break
            end
            for linea in eachline(top_file)
                # Found the atom names
                cnt+=1
                if cnt == nlines_an
                    resto = convert(Int64, floor(length(linea) / 4))
                    append!(atom_names_full, [ rstrip(linea[(4*(i-1)+1):(4*i)]) for i in 1:resto ])
                    break
                end
                atom_names_full[(20*(cnt-1)+1):(20*cnt)] = [ rstrip(linea[(4*(i-1)+1):(4*i)]) for i in 1:20 ]
            end
            break
        end
    end
    close(top_file)
    
    # One Array{Float64, 2} for each molecule, and the last one for water
    for i = 1:length(nnatoms_each_mol)
        lo_margin = natoms_each_mol[i] + 1
        try
            hi_margin = natoms_each_mol[i+1]
            atom_names[i] = atom_names_full[lo_margin:hi_margin]
        catch
            atom_names[i] = atom_names_full[lo_margin:end]
        end
    end

    
    
    return atom_names, sys_coords, sys_vels, bdims, bangles, nnatoms_each_mol, sum_natoms_all, time_ps
end

read_amber_crd (generic function with 4 methods)

In [9]:
function get_temp(vels_ap::Array{Float64, 2}, atom_names::Array{AbstractString, 1})
    const N = size(vels_ap)[2]
    const R_J = 8.314
    const amber_vel_scal = (20.455) * 100

    # Check input correctness
    if N != length(atom_names)
        error("Number of velocities and atom names don't match")
    end

    # Get the masses ready
    masses = Array{Float64}(N)
    for i = 1:length(atom_names)
        if atom_names[i][1] == 'C'
            masses[i] = 0.012
        elseif atom_names[i][1] == 'H'
            masses[i] = 0.001
        elseif atom_names[i][1] == 'O'
            masses[i] = 0.016
        elseif atom_names[i][1] == 'N'
            masses[i] = 0.014
        elseif atom_names[i][1] == 'S'
            masses[i] = 0.032
        else
            masses[i] = 0.012
            warn(string("Warning: element from atom ", i, " could not be determined. Assigning carbon mass."))
        end
    end

    # Get the speeds (norm of velocities) ready
    vels_ms = vels_ap * amber_vel_scal
    vels = mapslices(x -> sum(x.^2), vels_ms, 1)
    vels = reshape(vels, length(vels))

    # Get the system temperature
    const K = sum(0.5 .* masses .* vels)
    const T = K * (1/N) * (2/3) * (1/R_J)

    return T
end

get_temp (generic function with 1 method)

In [40]:
function write_rst7(filename_rst7::AbstractString, sys_coords::Array{Array{Float64, 2}}, sys_vels::Array{Array{Float64, 2}} = 0,
    time = 0, temp = 0, bdims::Array{Float64, 1} = 0, bangles::Array{Float64, 1} = 0)

    # Get the number of atoms in the system
    natoms = 0
    for i = 1:length(sys_coords)
        natoms = natoms + size(sys_coords[i])[2]
    end

    rst7_file = open(filename_rst7, "w")
    # Header
    @printf rst7_file "default_name                                                                \n"
    @printf rst7_file "%5i" natoms
    @printf rst7_file "%15.7E" time
    if temp != 0
        @printf rst7_file "%15.7E" temp
    end
    @printf rst7_file "\n" 

    # Coordinates
    for k = 1:length(sys_coords)
        for j in collect(1:size(sys_coords[k])[2])
            for i = 1:3
                @printf rst7_file "%12.7f" sys_coords[k][i, j]
            end
            if (j % 2) == 0
                @printf rst7_file "\n" 
                continue
            end
            if j == size(sys_coords[k])[2] && k == length(sys_coords)
                @printf rst7_file "\n" 
            end
        end
    end

    # Velocities
    if sys_vels != 0
        for k = 1:length(sys_vels)
            for j in collect(1:size(sys_vels[k])[2])
                for i = 1:3
                    @printf rst7_file "%12.7f" sys_vels[k][i, j]
                end
                if (j % 2) == 0
                    @printf rst7_file "\n" 
                end
            end
        end
    end

    # Box dimensions
    if bdims != 0
        @printf rst7_file "%12.7f" bdims[1]
        @printf rst7_file "%12.7f" bdims[2]
        @printf rst7_file "%12.7f" bdims[3]
    end
    # Box angles
    if bangles != 0
        @printf rst7_file "%12.7f" bangles[1]
        @printf rst7_file "%12.7f" bangles[2]
        @printf rst7_file "%12.7f" bangles[3]
    end
    @printf rst7_file "\n"


    close(rst7_file)
    return
end

write_rst7 (generic function with 6 methods)

## Lo usé p/ probar q mi cálculo de temp y el de cpptraj den iguales

home = "/home/german/labo/17/pdz/"
i = 1
### YO
in_coord_filename = string(home, "run/lb/neq/data/2/", i, "_lb.rst7")
in_top_filename = string(home, "top_files/lb.prmtop")

# Read original rst7
names_each, xyz, vel, box_dims, box_angles, natoms_each, natoms, time_ps = 
    read_amber_crd(in_top_filename, in_coord_filename);

# Change ligand temperature. Use all atoms to get ligand temperature
#indices = find(x -> x[1] != 'H', names_each[2])
lig_names = names_each[2][:]
lig_vel = vel[2][:, :]
mult = sqrt(3000 / get_temp(vel[2], names_each[2]))
#vel[2] = vel[2] .* mult

### Calculo temp del ligando con cpptraj, la aumento, y escribo los .rst7 con la temperatura del ligando a 3000K

In [41]:
home = "/home/german/labo/17/pdz/"

for i = 1:400
    # Get ready
    in_coord_filename = string(home, "run/lb/neq/data/2/", i, "plb_2.rst7")
    in_top_filename = string(home, "top_files/lb.prmtop")
    out_temp_filename = string("/home/german/labo/17/pdz/run/lb/neq/data/tjump/", i, "temp")
    
    # Read original rst7
    names_each, xyz, vel, box_dims, box_angles, natoms_each, natoms, time_ps = 
        read_amber_crd(in_top_filename, in_coord_filename);

    # Write cpptraj script to get ligand temperature
    cpp_filename = string("/home/german/labo/17/pdz/run/lb/neq/data/tjump/", i, "temp_cpp_in")
    cpp_file = open(cpp_filename, "w")

    texto = [ string("parm ", in_top_filename) ;
        string("trajin ", in_coord_filename) ;
        string("temperature T :125-132 ntc 1 out ", out_temp_filename) ; 
        string("go") ]
    writedlm(cpp_file, texto)
    
    close(cpp_file)
    
    # Run it
    run(`cpptraj -i $cpp_filename -o tmp`);
    
    # Read result
    out_temp_file = open(out_temp_filename, "r")
    lig_temp0 = convert(Float64, readdlm(out_temp_file)[1, 2])

    # Increase ligand temperature
    mult = sqrt(3000 / lig_temp0)
    vel[2] = vel[2] .* mult

    # Delete temporary (cpptraj script and its output file
    run(`rm $cpp_filename`);
    run(`rm tmp`);
    
    # Write out
    write_rst7(string(home, "run/lb/neq/data/tjump/T", i, "plb_2.rst7"), xyz, vel,
        time_ps, 0, box_dims, box_angles);
    
    # Show progress
    println(i)
end

Info: Redirecting output to file 'tmp'
1
Info: Redirecting output to file 'tmp'
2
Info: Redirecting output to file 'tmp'
3
Info: Redirecting output to file 'tmp'
4
Info: Redirecting output to file 'tmp'
5
Info: Redirecting output to file 'tmp'
6
Info: Redirecting output to file 'tmp'
7
Info: Redirecting output to file 'tmp'
8
Info: Redirecting output to file 'tmp'
9
Info: Redirecting output to file 'tmp'
10
Info: Redirecting output to file 'tmp'
11
Info: Redirecting output to file 'tmp'
12
Info: Redirecting output to file 'tmp'
13
Info: Redirecting output to file 'tmp'
14
Info: Redirecting output to file 'tmp'
15
Info: Redirecting output to file 'tmp'
16
Info: Redirecting output to file 'tmp'
17
Info: Redirecting output to file 'tmp'
18
Info: Redirecting output to file 'tmp'
19
Info: Redirecting output to file 'tmp'
20
Info: Redirecting output to file 'tmp'
21
Info: Redirecting output to file 'tmp'
22
Info: Redirecting output to file 'tmp'
23
Info: Redirecting output to file 'tmp'
24
I

194
Info: Redirecting output to file 'tmp'
195
Info: Redirecting output to file 'tmp'
196
Info: Redirecting output to file 'tmp'
197
Info: Redirecting output to file 'tmp'
198
Info: Redirecting output to file 'tmp'
199
Info: Redirecting output to file 'tmp'
200
Info: Redirecting output to file 'tmp'
201
Info: Redirecting output to file 'tmp'
202
Info: Redirecting output to file 'tmp'
203
Info: Redirecting output to file 'tmp'
204
Info: Redirecting output to file 'tmp'
205
Info: Redirecting output to file 'tmp'
206
Info: Redirecting output to file 'tmp'
207
Info: Redirecting output to file 'tmp'
208
Info: Redirecting output to file 'tmp'
209
Info: Redirecting output to file 'tmp'
210
Info: Redirecting output to file 'tmp'
211
Info: Redirecting output to file 'tmp'
212
Info: Redirecting output to file 'tmp'
213
Info: Redirecting output to file 'tmp'
214
Info: Redirecting output to file 'tmp'
215
Info: Redirecting output to file 'tmp'
216
Info: Redirecting output to file 'tmp'
217
Info: R

385
Info: Redirecting output to file 'tmp'
386
Info: Redirecting output to file 'tmp'
387
Info: Redirecting output to file 'tmp'
388
Info: Redirecting output to file 'tmp'
389
Info: Redirecting output to file 'tmp'
390
Info: Redirecting output to file 'tmp'
391
Info: Redirecting output to file 'tmp'
392
Info: Redirecting output to file 'tmp'
393
Info: Redirecting output to file 'tmp'
394
Info: Redirecting output to file 'tmp'
395
Info: Redirecting output to file 'tmp'
396
Info: Redirecting output to file 'tmp'
397
Info: Redirecting output to file 'tmp'
398
Info: Redirecting output to file 'tmp'
399
Info: Redirecting output to file 'tmp'
400


### Hago los scripts de cpptraj p/ calcular la temp de ligando a lo largo de las corridas corta y larga

# Cpp scripts p/ seguir la temperatura del ligando durante la corrida corta
for i = 1:400
    in_coord_filename = string("/scratch/pbarletta/pdz/run/lb/neq/data/corta_4/T", i, "plb_4.nc")
    in_top_filename = string("/scratch/pbarletta/pdz/top_files/lb.prmtop")
    out_temp_filename = string("/scratch/pbarletta/pdz/run/lb/neq/temperatura/lig/corta/", i, "corta_temp")
    
    # Write cpptraj script to get ligand temperature
    cpp_filename =
        string("/home/german/labo/17/pdz/run/lb/neq/temperatura/lig/corta/", i, "corta_temp_cpp_in")
    cpp_file = open(cpp_filename, "w")

    texto = [ string("parm ", in_top_filename) ;
        string("trajin ", in_coord_filename) ;
        string("temperature T :125-132 ntc 1 out ", out_temp_filename) ; 
        string("go") ]
    writedlm(cpp_file, texto)
    
    close(cpp_file)
end

# Cpp scripts p/ seguir la temperatura del ligando durante la corrida larga
for i = 1:400
    in_coord_filename = string("/scratch/pbarletta/pdz/run/lb/neq/data/larga_4/T", i, "pplb_4.nc")
    in_top_filename = string("/scratch/pbarletta/pdz/top_files/lb.prmtop")
    out_temp_filename = string("/scratch/pbarletta/pdz/run/lb/neq/temperatura/lig/larga/", i, "larga_temp")
    
    # Write cpptraj script to get ligand temperature
    cpp_filename = 
        string("/home/german/labo/17/pdz/run/lb/neq/temperatura/lig/larga/", i, "larga_temp_cpp_in")
    cpp_file = open(cpp_filename, "w")

    texto = [ string("parm ", in_top_filename) ;
        string("trajin ", in_coord_filename) ;
        string("temperature T :125-132 ntc 1 out ", out_temp_filename) ; 
        string("go") ]
    writedlm(cpp_file, texto)
    
    close(cpp_file)
end